In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load and preprocess the dataset
housing = fetch_california_housing()
X_train_full, X_test_full, y_train, y_test = train_test_split(housing.data, housing.target, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_full)
X_test = scaler.fit_transform(X_test_full)

In [3]:
# Building the TensorFlow Model
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(30, activation='relu'),
    
    
    
#     tf.keras.layers.Dense(600, activation='relu'),
    
    tf.keras.layers.Dense(1)
])

# Compiling the model
model_tf.compile(optimizer='sgd', loss='mean_squared_error')

# Training the TensorFlow model
model_tf.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=168)

Epoch 1/20


2024-01-04 15:45:44.541548: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


83/83 [==============================] - 1s 7ms/step - loss: 1.2100 - val_loss: 0.5958
Epoch 2/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5817 - val_loss: 0.5980
Epoch 3/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5389 - val_loss: 0.5504
Epoch 4/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5354 - val_loss: 0.5530
Epoch 5/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5289 - val_loss: 0.5485
Epoch 6/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5376 - val_loss: 0.5561
Epoch 7/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5223 - val_loss: 0.5471
Epoch 8/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5256 - val_loss: 0.5446
Epoch 9/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5302 - val_loss: 0.5491
Epoch 10/20
83/83 [==============================] - 0s 5ms/step - loss: 0.5255 - val_loss: 0.5529
Epoch 11/20
83/83 [===========

In [4]:
# Evaluate model
mse_test = model_tf.evaluate(X_test, y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.5327


In [5]:
import numpy as np

# Predictions in test sets
prediction = model_tf.predict(X_test)

162/162 [==============================] - 0s 2ms/step


In [6]:
# MSE 
mse = np.mean(np.square(prediction - y_test))
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 2.170025351128788


# PyTorch

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [8]:
# Transform train and test test to torch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

train_data = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [9]:
# Building the PyTorch model
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.linear1 = nn.Linear(X_train_torch.shape[1], 30)
        self.linear2 = nn.Linear(30, 1)
    
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        return self.linear2(x)

model_torch = RegressionModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model_torch.parameters(), lr=0.01)

In [10]:
# Training the PyTorch model
model_torch.train()
for epoch in range(20):
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model_torch(x_batch)
        loss = criterion(output, y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()

In [11]:
# Evaluate the PyTorch model
model_torch.eval()
with torch.no_grad():
    predictions = model_torch(X_test_torch)
    mse_test_torch = criterion(predictions, y_test_torch.unsqueeze(1)).item()

print("MSE Test (TensorFlow):", mse_test)
print("MSE Test (PyTorch):", mse_test_torch)

MSE Test (TensorFlow): 0.5326807498931885
MSE Test (PyTorch): 1.5519731044769287
